In [1]:
import pandas as pd
import sqlite3
import os
import numpy as np
import pickle
import time

In [3]:
# select year
year = 2014
trg = 'I50'
# trg = 'H26' #백내장

In [4]:
# open out_dict, that is, collection of all patients
with open('/home/data/EHR/db/out_dict_%d.pckl'%year,'rb') as f:
    out_dict = pickle.load(f)

In [5]:
out_dict[1]

[(4127696, 20140108.0, [1, 3, 636, 1143, 87], 636),
 (4899724, 20140129.0, [1, 3, 636, 1143, 87], 636),
 (5901835, 20140102.0, [1, 3, 636, 1143, 87], 636),
 (5994969, 20140108.0, [16, 611], 611),
 (6506747, 20140127.0, [1, 3, 636, 1143, 87], 636),
 (10420348, 20140104.0, [1, 859, 3, 621, 87], 621),
 (12492100, 20140124.0, [1, 3, 636, 1143, 87], 636),
 (16197789, 20140118.0, [2, 3, 612, 4, 1163, 909, 782, 886, 700], 700),
 (16751732, 20140114.0, [1, 3, 636, 1143, 87], 636),
 (20007209, 20140110.0, [1, 3, 636, 1143, 87], 636),
 (21179011, 20140128.0, [1, 3, 636, 1143, 87], 636),
 (21931920, 20140109.0, [1, 3, 636, 1143, 87], 636),
 (23008082, 20140116.0, [1, 3, 636, 1143, 87], 636),
 (24817466, 20140130.0, [1, 612, 776, 1001, 603, 859, 1180], 603),
 (24977894, 20140115.0, [1, 3, 636, 1143, 87], 636),
 (25187516, 20140123.0, [1, 3, 636, 1143, 87], 636),
 (25772942, 20140103.0, [1, 3, 636, 1143, 87], 636),
 (3389551, 20140227.0, [1, 3, 636, 1143, 87], 636),
 (6314661, 20140226.0, [1, 3, 63

In [26]:
# open database
con = sqlite3.connect('/home/data/EHR/db/%d.db'%year)
cur = con.cursor()
con.text_factory = str

In [27]:
cur.execute('SELECT spec_id FROM sick_table WHERE sick_ab IS "%s"'%trg)
# get all instances where I50 is observed
id_list = [x[0] for x in cur.fetchall()]

In [28]:
# preprocessing 1 - remove patients whose observation was conducted earlier than 6/31
jid_list = []
jid_remove_list = []
for id in id_list:
    cur.execute('SELECT jid,date FROM main_table WHERE spec_id is %d'%id)
    for (jid,date) in cur.fetchall():
        jid_list.append(jid) # add to jid list
        if date<(year*10000+631): # if the discovery was made 
            jid_remove_list.append(jid) # make sure we remove this later
jid_list = list(set(jid_list))
print(len(jid_list))
jid_remove_list = list(set(jid_remove_list))
print(len(jid_remove_list))
for jid in jid_remove_list:
    jid_list.remove(jid)
print(len(jid_list))

16594
12859
3735


In [29]:
# preprocessing 2 - remove patients with less than 5 visits earlier than 6/31
jid_remove_list = []
for jid in jid_list:
    try:
        if len(out_dict[jid])<5: # remove patients less than 5 visits
            jid_remove_list.append(jid)
            continue
        if out_dict[jid][4][1]>(year*10000+631): # if a patient has less than 5 visits beforehand, remove as well
            jid_remove_list.append(jid)
            continue
    except KeyError:
        continue
for jid in jid_remove_list:
    jid_list.remove(jid)
print(len(jid_list))

2890


In [31]:
# for each positive jid, save all visits that take place up to 6/31, and store it to out_dict2
out_dict2 = dict()
for jid in jid_list:
    cnt = 0
    try:
        for tup in out_dict[jid]:
            if tup[1]>(year*10000+631):
                break
            cnt+=1
        out_dict2[jid]=out_dict[jid][:cnt]
    except KeyError:
        continue

In [32]:
# save positive outputs to out_pos in /data/preprocessed/I50
with open('../data/preprocessed/%s/out_pos_%d.pckl'%(trg,year),'wb') as f:
    pickle.dump(out_dict2,f)

In [33]:
with open('../data/preprocessed//%s/out_pos_%d.pckl'%(trg,year),'rb') as f:
    out_dict2 = pickle.load(f)

In [34]:
pos_list = list(out_dict2.keys())
print(len(pos_list))

2828


In [35]:
# count number of visits for each patient prior to 0631
visit_dict = dict()
for k,V in out_dict.items():
    cnt = 0
    for (_,date,_,_) in V:
        if date>(year*10000+631):
            break
        cnt+=1
    visit_dict[k]=cnt
with open('../data/preprocessed/%s/visits_%d.pckl'%(trg,year),'wb') as f:
    pickle.dump(visit_dict,f)

In [36]:
# exclude list - patients with at least one diagnosis in I50
exclude_list = []
cur.execute('SELECT spec_id FROM sick_table WHERE sick_ab IS "%s"'%trg) # get all instances where I50 is observed
id_list = [x[0] for x in cur.fetchall()]
for id in id_list:
    cur.execute('SELECT jid FROM main_table WHERE spec_id is %d'%id)
    for (jid,) in cur.fetchall():
        exclude_list.append(jid) # add to jid list
exclude_list = list(set(exclude_list)-set(pos_list))
candidate_list = list(set(out_dict.keys())-set(exclude_list))
print(len(pos_list))
print(len(candidate_list))
print(len(exclude_list))

2828
1417434
13766


In [37]:
with open('/home/data/EHR/db/%d_demo.pckl' %(year),'rb') as f:
    D = pickle.load(f)

In [38]:
# D.sort_values('jid')

In [39]:
D2 = D[D['jid'].isin(candidate_list)]

In [40]:
# cur.execute('DROP TABLE demo_table_tmp')
# con.commit()

In [41]:
D2.to_sql('demo_table_tmp',con=con)
cur.execute('CREATE INDEX idx_demo_1 ON demo_table_tmp(jid)')
cur.execute('CREATE INDEX idx_demo_2 ON demo_table_tmp(agg,gender)')
con.commit()

In [42]:
# df_demo.to_sql('demo_table',con=con)
# cur.execute('CREATE INDEX idx_demo ON demo_table(jid)')
# cur.execute('CREATE INDEX idx_demo2 ON demo_table(agg,gender)')
# con.commit()

In [43]:
import time
start = time.time()
out_list = []
exception_list = []
cnt = 0
for jid in pos_list:
    length = visit_dict[jid]
    cnt+=1
    if cnt%100==0:
        print(cnt)
        print(time.time()-start)
    cur.execute('SELECT agg,gender FROM demo_table_tmp WHERE jid IS %d' %jid)
    out = cur.fetchone()
    cur.execute('SELECT jid FROM demo_table_tmp WHERE agg IS %d AND gender IS %d' %out)
    results = list(set([x[0] for x in cur.fetchall()]))
    neg = []
    for jid2 in results:
        try:
            len2 = visit_dict[jid2]
            if (len2>=length) & (len2<=length*1.3):
                neg.append(jid2)
            if len(neg)>=11:
                break  
        except KeyError:
            continue
    if jid in neg:
        neg.remove(jid)
    else:
        neg = neg[:10]
    if len(neg)==10:
        out_list.append((jid,neg))
    else:
        exception_list.append((jid,len(neg)))

100
5.1197896003723145
200
10.517198085784912
300
15.708556890487671
400
20.695252895355225
500
25.872736930847168
600
30.905836582183838
700
36.094778537750244
800
41.123481035232544
900
46.30752491950989
1000
51.6274197101593
1100
57.15680956840515
1200
62.36011981964111
1300
67.73951268196106
1400
73.24860572814941
1500
78.3967535495758
1600
83.77292823791504
1700
88.88983225822449
1800
94.12806940078735
1900
99.30260133743286
2000
104.32634472846985
2100
109.56003475189209
2200
114.71899104118347
2300
119.8372917175293
2400
124.82301354408264
2500
130.27404618263245
2600
135.57473945617676
2700
140.8736503124237
2800
146.4322154521942


In [44]:
len(exception_list)

2

In [45]:
out_list2 = []
answers = list(np.zeros(11,dtype=int))
answers[0] = 1
for (pos,neg_list) in out_list:
    visit_len = visit_dict[pos]
    input_list = []
    target_list = list(np.zeros(len(neg_list)+1,dtype=int))
    target_list[0] = 1
    input_list.append(out_dict[pos][:visit_len])
    for neg in neg_list:
        input_list.append(out_dict[neg][:visit_len])
    out_list2.append((input_list,target_list))

In [46]:
with open('../data/preprocessed/%s/list_data_%d.pckl'%(trg,year),'wb') as f:
    pickle.dump(out_list2,f)

In [47]:
cur.execute('DROP INDEX idx_demo_1')
cur.execute('DROP INDEX idx_demo_2')
cur.execute('DROP TABLE demo_table_tmp')
con.commit()

In [48]:
con.close()